In [ ]:
!git clone https://github.com/OpenNMT/OpenNMT-tf.git

In [ ]:
pip install OpenNMT-tf[tensorflow_gpu]

Run below cell only if Back-Translation is required

In [ ]:
# Train a Reverse Model (Target to Source translation) for generating data using back-translation
!onmt-main --config data.yml --auto_config infer --features_file '/content/gdrive/My Drive/Eng-Hindi/en_news_BT' --predictions_file '/content/gdrive/My Drive/Eng-Hindi/Hardik_Model/eval/en_news_BT.txt' # To load the model and use it to predict the output 
!onmt-main --config data.yml --auto_config infer --features_file 'cleaned_test.txt' --predictions_file 'cleaned_test_pred_bpe.txt'

Only run the below cell if Using BPE

In [ ]:
#Learn and Apply Byte Pair Encoding using Train, Test and Dev Data (Both English and Hindi)
!python OpenNMT-tf/third_party/learn_bpe.py -i "/content/gdrive/My Drive/Eng-Hindi/cleaned_train_en" -o "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/tgt.code" -s 27000
!python OpenNMT-tf/third_party/learn_bpe.py -i "/content/gdrive/My Drive/Eng-Hindi/cleaned_train_hi" -o "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/src.code" -s 27000
!python OpenNMT-tf/third_party/apply_bpe.py -c  "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/tgt.code" -i  "/content/gdrive/My Drive/Eng-Hindi/cleaned_train_en" -o "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/tgt-train-bpe.txt"
!python OpenNMT-tf/third_party/apply_bpe.py -c  "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/tgt.code" -i  "/content/gdrive/My Drive/Eng-Hindi/cleaned_dev_en" -o "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/tgt-val-bpe.txt"
!python OpenNMT-tf/third_party/apply_bpe.py -c  "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/tgt.code" -i  "/content/gdrive/My Drive/Eng-Hindi/cleaned_test_en_onlysametokenintrain.txt" -o "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/tgt-test-bpe.txt"
!python OpenNMT-tf/third_party/apply_bpe.py -c "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/src.code" -i "/content/gdrive/My Drive/Eng-Hindi/cleaned_train_hi" -o "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/src-train-bpe.txt"
!python OpenNMT-tf/third_party/apply_bpe.py -c "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/src.code" -i "/content/gdrive/My Drive/Eng-Hindi/cleaned_dev_hi" -o "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/src-val-bpe.txt"
!python OpenNMT-tf/third_party/apply_bpe.py -c "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/src.code" -i "/content/gdrive/My Drive/Eng-Hindi/cleaned_test_hi_onlysametokenintrain.txt" -o "/content/gdrive/My Drive/Eng-Hindi/Hindi_Eng_Model/src-test-bpe.txt"

In [ ]:
#For creating Source Vocabulary using Byte Pair Encoding (BPE) for Train, Test and Dev Data
!onmt-build-vocab --size 50000 --save_vocab "/content/gdrive/My Drive/Eng-Hindi/Hardik_Model/tgt-train-vocab.txt" "/content/gdrive/My Drive/Eng-Hindi/cleaned_train_hi"
!onmt-build-vocab --size 50000 --save_vocab "/content/gdrive/My Drive/Eng-Hindi/Hardik_Model/tgt-dev-vocab.txt" "/content/gdrive/My Drive/Eng-Hindi/cleaned_dev_hi"
!onmt-build-vocab --size 50000 --save_vocab "/content/gdrive/My Drive/Eng-Hindi/Hardik_Model/tgt-test-vocab.txt" "/content/gdrive/My Drive/Eng-Hindi/cleaned_test_hi"

#For creating Target Vocabulary using Byte Pair Encoding (BPE) for Train, Test and Dev Data
!onmt-build-vocab --size 50000 --save_vocab "/content/gdrive/My Drive/Eng-Hindi/Hardik_Model/src-train-vocab.txt" "/content/gdrive/My Drive/Eng-Hindi/cleaned_train_en"
!onmt-build-vocab --size 50000 --save_vocab "/content/gdrive/My Drive/Eng-Hindi/Hardik_Model/src-dev-vocab.txt" "/content/gdrive/My Drive/Eng-Hindi/cleaned_dev_en"
!onmt-build-vocab --size 50000 --save_vocab "/content/gdrive/My Drive/Eng-Hindi/Hardik_Model/src-test-vocab.txt" "/content/gdrive/My Drive/Eng-Hindi/cleaned_test_en"

In [ ]:
#Training the Transformer Model
#data.yml consists of all run configurations
!onmt-main --model_type Transformer --config data.yml --auto_config train --with_eval

In [ ]:
#download Multi-bleu perl for determining the BLEU score
!sed -i "s/@@ //g" "predictions.70000.txt"
!perl  multi-bleu-detok.perl "cleaned_test_en_onlysametokenintrain.txt" < "predictions.70000.txt"

BLEU = 23.45, 58.2/30.8/17.3/10.1 (BP=0.992, ratio=0.992, hyp_len=29670, ref_len=29899)
